In [1]:
import boto3 
import sagemaker
from ibov.utils import load_config
from sagemaker.pytorch import PyTorch

### Loading Configs

In [2]:
# Load config dict
config = load_config()

In [3]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

### Upload Data

In [4]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [5]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [13]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge')

In [14]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-14 02:20:19 Starting - Starting the training job...
2021-01-14 02:20:44 Starting - Launching requested ML instancesProfilerReport-1610590817: InProgress
......
2021-01-14 02:21:45 Starting - Preparing the instances for training.........
2021-01-14 02:23:37 Downloading - Downloading input data
2021-01-14 02:23:37 Training - Downloading the training image...
2021-01-14 02:24:07 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-14 02:24:04,138 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-14 02:24:04,172 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-14 02:24:07,198 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-14 02:24:07,476 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating set


2021-01-14 02:24:40 Uploading - Uploading generated training model
2021-01-14 02:24:40 Failed - Training job failed
ProfilerReport-1610590817: Stopping


UnexpectedStatusException: Error for Training job sagemaker-pytorch-2021-01-14-02-20-15-756: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python -m train"
Traceback (most recent call last):
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/ml/code/train.py", line 14, in <module>
    from request import label_train_test
  File "/opt/ml/code/request.py", line 1, in <module>
    from yahooquery import Ticker
ImportError: No module named 'yahooquery'

predictor = estimator.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge")